## This can be used for preprocessing of both sentiment analysis and categorical classification. 

#### Just change the necessary paths and parts of code for 2 different types of datasets

In [ ]:
import sys
import regex as re
from tqdm import tqdm_notebook as tqdm
import pandas as pd

FLAGS = re.MULTILINE | re.DOTALL

def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = "<hashtag> {} <allcaps>".format(hashtag_body.lower())
    else:
        result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
    return result

def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps> " # amackcrane added trailing space


def tokenize(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ")
    text = re_sub(r"@\w+", " <user> ")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), " <smile> ")
    text = re_sub(r"{}{}p+".format(eyes, nose), " <lolface> ")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), " <sadface> ")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), " <neutralface> ")
    text = re_sub(r"/"," / ")
    text = re_sub(r"<3"," <heart> ")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ")
    text = re_sub(r"#\w+", hashtag)  # amackcrane edit
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat> ")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong> ")
    

    

    # amackcrane additions
    text = re_sub(r"([a-zA-Z<>()])([?!.:;,])", r"\1 \2")
    text = re_sub(r"\(([a-zA-Z<>]+)\)", r"( \1 )")
    text = re_sub(r"  ", r" ")
    text = re_sub(r" ([A-Z]){2,} ", allcaps)
    
    return text.lower()

def is_allowed_specific_char(string):
    charRe = re.compile(r'[^a-zA-Z@:\\]')
    string = charRe.search(string)
    return not bool(string)

#_, text = sys.argv  # kaggle envt breaks this -amackcrane
#if text == "test":
text = "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
text2 = "'RT @nsg_ram: @GDGNITSurat @GoogleCloud #GoogleDevelopersGroup #Next17\\rWhat good is watching a movie without popcorn \\rAnd A Seminar with foo…'" # couple extra tests -amackcrane
tokens = tokenize(text)
print(tokens)
print(tokenize(text2).replace("\\r","").replace("'","").replace('"',""))



In [1]:
from sklearn.utils import shuffle
import numpy as np
from tqdm import tqdm_notebook as tqdm

### Give path for train_data.csv that you downloaded from Stanford site

dataset = pd.read_csv('drive/My Drive/trainingandtestdata/train_data.csv',encoding='latin-1')

dataset = shuffle(dataset)
print(dataset)



dataset.columns = ["polarity","r1","r2","r3","r4","tweet"]
dataset = dataset.drop(columns = ["r1","r2","r3","r4"])
dataset=dataset
print(dataset.info)



tweets = dataset["tweet"]
tweets = np.array( tweets )
n = len( tweets )
print(n)



for i in tqdm(range(dataset.shape[0])):
  tweets[i] = tokenize(tweets[i])
  tweets[i] = re.sub(r"[^'<>\w\s]", " " , tweets[i])
  tweets[i] = re.sub(r"[']", "" , tweets[i])
  tweets[i] = ' '.join(tweets[i].split())
    


dataset['tweet'] = tweets
dataset.insert(0, 'index', range(len(all_data)))
print(dataset)


'''
Include below code for categorical classification dataset

all_data["category"] = all_data["category"].replace("ENTERTAINMENT","0")
all_data["category"] = all_data["category"].replace("POLITICS","1")
all_data["category"] = all_data["category"].replace("SPORTS","2")
all_data["category"] = all_data["category"].replace("BUSINESS","3")
all_data["category"] = all_data["category"].replace("EDUCATION","4")

'''



#### Give appropriate path ####
#### Write "polarity" or "category" in as middle column as required ####


dataset.to_csv('drive/My Drive/trainingandtestdata/glove_processed_data.csv', index=False, sep="|", header=["index","polarity","tweet"])

NameError: name 'pd' is not defined